## Cashtag Counts

The following script reveals cashtag mentions on Twitter over time.

#### Outcome 
* Results folder that contains a .csv file for each cashtag
* Charts folder that contains a graph for each cashtag

In [1]:
import os
import sys
import yaml
import json
import requests
from requests_oauthlib import OAuth1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.dates as mdates
import seaborn as sns

In the root directory, create a new file entitled `credentials.yaml` and populate the information (as exemplified in `credentials.yaml.template`). The credentials must be tied to Twitter's Search API (Full Archive) at enterprise access. 

In [2]:
with open(r'./credentials.yaml') as file:
    credentials = yaml.load(file, Loader=yaml.FullLoader)
    
    username = credentials['enterprise_full_archive_search_credentials']['username']
    password = credentials['enterprise_full_archive_search_credentials']['password']
    account_name = credentials['enterprise_full_archive_search_credentials']['account_name']
    env_label = credentials['enterprise_full_archive_search_credentials']['env_label']

In [3]:
endpoint = f'https://gnip-api.twitter.com/search/fullarchive/accounts/{account_name}/{env_label}/counts.json'

The following cell is where you can specify the data you want to get back: 
* What stock you want to track (e.g. TWTTR) 
* What time period you'd like to pull data for (format: YYYYMMddHHmm)
* What time bucket you want to use (options: 'day', 'hour', 'minute')

In [4]:
stock = 'TWTR'
fromDate = '202001010000'
toDate = '202103040000'
bucket = 'hour'

# Only change parameters above this line 

body = {
    'query': f'${stock}', 
    'fromDate': f'{fromDate}',
    'toDate': f'{toDate}',
    'bucket': f'{bucket}'
}

In [ ]:
response = requests.post(url=endpoint, auth=(username, password), json=body)
data = response.json()

timePeriods = []
counts = []
if 'error' in data:
    print(data)
else:
    totalCount = data['totalCount']
    print("Activity count:", totalCount)
    for result in data['results']:
        timePeriods.append(result['timePeriod'])
        counts.append(result['count'])

In [ ]:
request_count = 1
while 'next' in data.keys():
    next_token = data['next']
    body.update(next=next_token)
    try: 
        response = requests.post(url=endpoint, auth=(username, password), json=body)
        data = response.json()
        print("Activity count:", data['totalCount'])
        totalCount += data['totalCount']
        for result in data['results']:
            timePeriods.append(result['timePeriod'])
            counts.append(result['count'])
    except requests.exceptions.RequestException as e:
        print(e)
    request_count += 1
    data = response.json()

print(f'Job complete: {request_count} request(s) in total')
print('Total activity count:', totalCount)

In [ ]:
df = pd.DataFrame({'Time period':timePeriods, 'Count':counts})

In [ ]:
df['Time period'] = pd.to_datetime(df['Time period'])

In [ ]:
df.head()

In [ ]:
df.sort_values(by='Time period')

In [ ]:
df.to_csv(f'Results/{stock}_counts.csv')

In [ ]:
df = df.sort_values(by='Time period')

params = {
    'axes.labelsize' : 16,
    'xtick.labelsize' : 12,
    'ytick.labelsize' : 12,
    'figure.figsize' : (20,10)
}
pylab.rcParams.update(params)

fig, ax = plt.subplots()
ax.plot(df['Time period'], df['Count'], lw=1, ls='-', c='#1DA1F2', marker='o', ms=0)
fig.autofmt_xdate()

# For smaller datasets - display labels on x axis by day
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%y')) 
# ax.xaxis.set_major_locator(mdates.DayLocator(interval=5))

# For larger datasets - display labels on x axis by month
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%y')) 
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))

ax.set_xlabel("Time period")
ax.set_ylabel("Count")

plt.savefig(f'Charts/{stock}', dpi=300)